In [131]:
using LinearAlgebra
using PrettyTables

In [132]:
# Function to approximate the leading eigenvalue of a matrix using the QR method
function qr_method(A, max_iter)
    # Initialize A_k with the input matrix A
    A_k = copy(A)

    # Iterate up to a maximum number of iterations
    for k = 1:max_iter
        # Perform QR decomposition on A_k
        Q, R = qr(A_k)
        # Form the next matrix A_k by multiplying R and Q
        A_k = R * Q
    end

    # # Return the leading eigenvalue, which is the first diagonal element of A_k
    return A_k[1,1]
end

qr_method (generic function with 1 method)

In [133]:
# Define a function to compute the leading eigenvalue using the Power Method
function power_method(A, max_iter)
    # Determine the size of the matrix A and initialize a random vector of this size 
    n, _ = size(A)
    v = rand(n)  # Start with a random vector
    v = v / norm(v)  # Normalize the vector

    # Iterate for a maximum number of iterations or until convergence
    for _ = 1:max_iter
        # Multiply the vector by the matrix A
        v_new = A * v
        # Normalize the resulting vector
        v_new = v_new / norm(v_new)
        # Update the vector for the next iteration
        v = v_new
    end

    # Calculate the leading eigenvalue which is done by the Rayleigh quotient: (v'Av) / (v'v)
    eigenvalue = dot(A * v, v) / dot(v, v)
    return eigenvalue, v
end

power_method (generic function with 1 method)

In [134]:
# Function to find all eigenvalues and eigenvectors of a symmetric matrix using deflation
function deflation(A, max_iter)
    k = size(A, 1)
    eigenvalues = []
    eigenvectors = []

    # Main loop for finding all eigenvalues and eigenvectors
    for i = 1:k
        eigenvalue, eigenvector = power_method(A, max_iter)
        push!(eigenvalues, eigenvalue)
        push!(eigenvectors, eigenvector)
        A = A - eigenvalue * eigenvector * transpose(eigenvector)
    end

    return eigenvalues, eigenvectors
end

deflation (generic function with 1 method)

In [135]:
#Function to obtain all the singular values and singular vectors
function singular_values_vectors(A, max_iter)

v_eigenvalue, v_eigenvector = power_method(transpose(A) * A, max_iter)
singular_values, v_singular_vectors = deflation(transpose(A) * A, max_iter)
singular_values = sqrt.(singular_values)

u_eigenvalue, u_eigenvector = power_method(A * transpose(A), max_iter)
_, u_singular_vectors = deflation(A * transpose(A), max_iter)

return singular_values, v_singular_vectors, u_singular_vectors
end

singular_values_vectors (generic function with 1 method)

In [136]:
#Function to generate a random symmetric matrix A and A^TA
function rand_matrix_generator(size, min_val, max_val)
    X = rand(min_val : max_val, size, size)
    A = transpose(X) * X
    return A
end 

rand_matrix_generator (generic function with 1 method)

In [137]:
#Function to compute and display all the eigenvalues and eigenvectors
function display_computations(A, max_iter_qr, max_iter_pm)
    #get leading eigenvalue from QR method
    eigenvalue_qr = qr_method(A, max_iter_qr)
    print("A's leading eigenvalue (QR method)")
    display(eigenvalue_qr)

    #get leading eigenvector and eigenvalue from the power method
    eigenvalue_pm, eigenvector_pm = power_method(A, max_iter_pm)
    print("A's leading eigenvector (Power method):")
    display(eigenvector_pm)
    print("A's leading eigenvalue (Power method and Raleigh quotient):")
    display(eigenvalue_pm)

    #get all eigenvalues and eigenvectors
    eigenvalues, eigenvectors = deflation(A, max_iter_pm)
    println("A's eigenvectors and corresponding eigenvalues (Deflation):")
    for i in range(1, length(eigenvectors))
        print("Eigenvector number " * string(i) * " is:")
        display(eigenvectors[i, 1])
        println("And has corresponding eigenvalue:")
        display(eigenvalues[i, 1])
        println("")
    end

    #get all singular vectors and singular values
    singular_values, left_singular_vectors, right_singular_vectors = singular_values_vectors(A, max_iter_pm)
    println("A's singular vectors for matrix V in A's SVD are:")
    for i in range(1, length(left_singular_vectors))
        println("Singular vector number " * string(i) * " is:")
        display(left_singular_vectors[i, 1])
    end    
    println("A's singular vectors for matrix U in A's SVD are:")
    for i in range(1, length(right_singular_vectors))
        println("Singular vector number " * string(i) * " is:")
        display(right_singular_vectors[i, 1])
    end
    println("A's singular values are:")
    display(singular_values)

    println("-----------------------------------------------------")
    println("Julia's Output to compare")
    display(eigvals(A))
    display(eigvecs(A))
    display(svd(A))
    end

display_computations (generic function with 1 method)

In [139]:
#Running our functions
A = rand_matrix_generator(8, 1, 14)
display_computations(A, 11, 11)

A's leading eigenvalue (QR method)

3706.6197377438066

A's leading eigenvector (Power method):

8-element Vector{Float64}:
 0.3513608224753594
 0.37638183990226576
 0.3237371058940461
 0.32864052652044595
 0.28750607067702877
 0.2945645790232591
 0.3102680156972951
 0.5063375358008766

A's leading eigenvalue (Power method and Raleigh quotient):

...

8-element Vector{Float64}:
 3706.619737743812
  380.9711771879006
  316.44171865432446
  185.82225295647052
   74.94671524695414
   45.272401503073965
   23.701242414976036
   13.225752844573659

-----------------------------------------------------
Julia's Output to compare


8-element Vector{Float64}:
   13.225752844526445
   23.70124241494957
   45.272401501636026
   74.94671524727146
  185.8222528779295
  316.4393799764263
  380.97251739344813
 3706.619737743812

8×8 Matrix{Float64}:
  0.393879   -0.602221   0.501589   …   0.262792    0.133212   -0.351361
 -0.0918407   0.310483   0.236862      -0.240159    0.615666   -0.376382
 -0.436076    0.164185   0.123375       0.581135   -0.0644282  -0.323737
 -0.224224   -0.22732   -0.635385       0.138846    0.421613   -0.328641
  0.171716   -0.049901  -0.0558159     -0.611157    0.0446528  -0.287506
 -0.621588   -0.176439   0.282419   …  -0.359027   -0.403529   -0.294565
  0.129069   -0.332191  -0.432563      -0.0356102  -0.368613   -0.310268
  0.404314    0.564205  -0.0581619      0.112194   -0.347271   -0.506338

SVD{Float64, Float64, Matrix{Float64}, Vector{Float64}}
U factor:
8×8 Matrix{Float64}:
 -0.351361   0.133212    0.262792   …   0.501589   -0.602221   0.393879
 -0.376382   0.615666   -0.240159       0.236862    0.310483  -0.0918407
 -0.323737  -0.0644282   0.581135       0.123375    0.164185  -0.436076
 -0.328641   0.421613    0.138846      -0.635385   -0.22732   -0.224224
 -0.287506   0.0446528  -0.611157      -0.0558159  -0.049901   0.171716
 -0.294565  -0.403529   -0.359027   …   0.282419   -0.176439  -0.621588
 -0.310268  -0.368613   -0.0356102     -0.432563   -0.332191   0.129069
 -0.506338  -0.347271    0.112194      -0.0581619   0.564205   0.404314
singular values:
8-element Vector{Float64}:
 3706.6197377438125
  380.9725173934479
  316.4393799764264
  185.8222528779295
   74.9467152472715
   45.27240150163626
   23.70124241494977
   13.225752844526577
Vt factor:
8×8 Matrix{Float64}:
 -0.351361  -0.376382   -0.323737   …  -0.294565  -0.310268   -0.506338
  0.133212   0.615666   

In [140]:
A_QR = [[7 1]; [1 7]]

2×2 Matrix{Int64}:
 7  1
 1  7

In [141]:
# Function to approximate the leading eigenvalue of a matrix using the QR method (example)
function qr_method_example(A, max_iter)
    # Initialize A_k with the input matrix A
    A_k = copy(A)

    # Iterate up to a maximum number of iterations
    for k = 1:max_iter
        # Perform QR decomposition on A_k
        Q, R = qr(A_k)
        print("Q: ")
        display(Q)
        print("R: ")
        display(R)
        # Form the next matrix A_k by multiplying R and Q
        A_k = R * Q
    end

    print("The leading eigevalue of A is: ")
    # # Return the leading eigenvalue, which is the first diagonal element of A_k
    display(A_k[1,1])
    return A_k[1,1]
end

qr_method_example (generic function with 1 method)

In [142]:
qr_method_example(A_QR, 5)

Q: 

2×2 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}:
 -0.989949  -0.141421
 -0.141421   0.989949

R: 

2×2 Matrix{Float64}:
 -7.07107  -1.9799
  0.0       6.78823

Q: 

...

R: 

2×2 Matrix{Float64}:
 -7.83913  -1.0273
  0.0       6.12313

The leading eigevalue of A is: 

7.893377271188352

7.893377271188352

In [143]:
A_Power = [[7 3 5]; [3 5 1]; [5 1 1]]

3×3 Matrix{Int64}:
 7  3  5
 3  5  1
 5  1  1

In [144]:
# Define a function to compute the leading eigenvalue using the Power Method (Example)
function power_method_example(A, max_iter)
    # Determine the size of the matrix A and initialize a random vector of this size 
    n, _ = size(A)
    v = rand(n)  # Start with a random vector
    v = v / norm(v)  # Normalize the vector

    # Iterate for a maximum number of iterations or until convergence
    for _ = 1:max_iter
        # Multiply the vector by the matrix A
        v_new = A * v        
        print("The updated vector is ")
        display(v_new)
        # Normalize the resulting vector
        v_new = v_new / norm(v_new)
        # Update the vector for the next iteration
        v = v_new
    end

    # Calculate the leading eigenvalue which is done by the Rayleigh quotient: (v'Av) / (v'v)
    eigenvalue = dot(A * v, v) / dot(v, v)
    print("The leading eigenvalue is ")
    display(eigenvalue)
    print("The leading eigenvector is ")
    display(v)
    return eigenvalue, v
end

power_method_example (generic function with 1 method)

In [153]:
power_method_example(A_Power, 5)

The updated vector is 

3-element Vector{Float64}:
 8.161030539868374
 4.627094182971236
 3.3134414620023938

The updated vector is 

3-element Vector{Float64}:
 8.802075512800766
 5.119083565097348
 4.899340934828883

The updated vector is 

...

The leading eigenvalue is 

11.35345127702423

The leading eigenvector is 

3-element Vector{Float64}:
 0.7909951454764633
 0.4406971595526109
 0.42439685837071095

(11.35345127702423, [0.7909951454764633, 0.4406971595526109, 0.42439685837071095])

In [145]:
A_Deflation = [[7 3 5]; [3 5 1]; [5 1 1]]

3×3 Matrix{Int64}:
 7  3  5
 3  5  1
 5  1  1

In [156]:
# Function to find all eigenvalues and eigenvectors of a symmetric matrix using deflation (Example)
function deflation_example(A, max_iter)
    k = size(A, 1)
    eigenvalues = []
    eigenvectors = []

    # Main loop for finding all eigenvalues and eigenvectors
    for i = 1:k
        eigenvalue, eigenvector = power_method(A, max_iter)
        push!(eigenvalues, eigenvalue)
        push!(eigenvectors, eigenvector)
        A = A - eigenvalue * eigenvector * transpose(eigenvector)
        print("The new matrix is ")
        display(A)
    end

    print("The list of eigenvalues is")
    for i = 1:size(eigenvalues, 1)
        display(eigenvalues[i])
    end
    print("The list of eigenvectors is")
    for i = 1:size(eigenvectors, 1)
        display(eigenvectors[i])
    end
    return eigenvalues, eigenvectors
end

deflation_example (generic function with 1 method)

In [157]:
deflation_example(A_Deflation, 5)

The new matrix is 

3×3 Matrix{Float64}:
 -0.107028  -0.953945   1.18611
 -0.953945   2.80025   -1.12183
  1.18611   -1.12183   -1.04667

The new matrix is 

3×3 Matrix{Float64}:
 -0.149139  -0.680337   0.981404
 -0.680337   1.02252    0.208205
  0.981404   0.208205  -2.04176

The new matrix is 

...

The list of eigenvectors is

3-element Vector{Float64}:
 0.7911886727089043
 0.4401721932271454
 0.42458088097172625

3-element Vector{Float64}:
 -0.12230960774710155
  0.7946934140708866
 -0.5945609619585772

3-element Vector{Float64}:
  0.40910363121697507
  0.13622557702085586
 -0.9022620523386873

(Any[11.353452085530611, 2.8149303216918904, -2.5196568848768055], Any[[0.7911886727089043, 0.4401721932271454, 0.42458088097172625], [-0.12230960774710155, 0.7946934140708866, -0.5945609619585772], [0.40910363121697507, 0.13622557702085586, -0.9022620523386873]])